In [35]:
# Import Splinter and BeautifulSoup
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import time 
import re

In [36]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


## Visit the NASA mars news site

In [37]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [38]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [39]:
news_title = soup.find('div', class_='content_title').text
print(news_title)

Mars Now


In [40]:
news_p = soup.find('div', class_='article_teaser_body').text
print(news_p)

Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.


## JPL Space Images Featured Image

In [41]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [42]:
html = browser.html
jpl_soup = BeautifulSoup(html, 'html.parser')

In [43]:
featured_image_url = jpl_soup.find('div',class_='carousel_items').a['data-fancybox-href']
print(featured_image_url)

/spaceimages/images/mediumsize/PIA17900_ip.jpg


In [44]:
featured_image_url = ('https://www.jpl.nasa.gov/' + featured_image_url)

In [45]:
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA17900_ip.jpg


## Mars Weather

In [46]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(5)

In [47]:
html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

In [48]:
mars_weather_tweets = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [49]:
try:
    mars_weather = mars_weather_tweets.find("p", "tweet-text").get_text()
    mars_weather

except AttributeError:

    pattern = re.compile(r'sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    mars_weather

mars_weather

'InSight sol 498 (2020-04-21) low -94.3ºC (-137.7ºF) high -5.7ºC (21.8ºF)\nwinds from the SW at 5.0 m/s (11.3 mph) gusting to 16.6 m/s (37.2 mph)\npressure at 6.60 hPa'

## Mars Hemispheres

In [59]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [60]:
html_hemispheres = browser.html
soup = BeautifulSoup(html_hemispheres, 'html.parser')


In [65]:
items = soup.find_all('div', class_='item')

hemisphere_image_urls = []

hemispheres_main_url = 'https://astrogeology.usgs.gov'


for i in items: 
    title = i.find('h3').text
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    browser.visit(hemispheres_main_url + partial_img_url)
    partial_img_html = browser.html
    soup = BeautifulSoup( partial_img_html, 'html.parser')

img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    
browser.back()

hemisphere_image_urls

[{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


## Mars Facts

In [53]:
facts_url = "https://space-facts.com/mars/"

In [54]:
pd_tables = pd.read_html(facts_url)

pd_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
                       0                              1
 0             Diameter:                       3,475 km
 1                 Mass:  7.35 × 10^22 kg (0.01 Earths)
 2               Orbits:                      The Earth
 3       Orbit Distance:                     384,400 km
 4         Orbit Period:                      27.3 days
 5  Surface Temperature:                 -233 to 123 °C,
   Mars - Earth Comparison             Mars   

In [55]:
mars_df = pd_tables[0]
mars_df.columns = ['Fact', 'Value']
mars_df.head()

,Fact,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [58]:
facts_html_table = mars_df.to_html()
facts_html_table = facts_html_table.replace('\n', '')
facts_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Fact</th>      <th>Value</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td

In [111]:
browser.quit()